In [1]:
# external imports
import music21 as m21
import ast
import os
import math

In [18]:
# reload cell
from src.midi import *
from src.train import *
import sys, importlib
importlib.reload(sys.modules['src.midi'])
importlib.reload(sys.modules['src.train'])

<module 'src.train' from 'D:\\Documents\\GitHub\\Pyotr\\src\\train.py'>

# Encoding and Read-in

In [3]:
# Read in all of the midi files
path = './data/sample'
mdl = gen_md_from_path(path, by_measure=False, verbose=False)
mdm = gen_md_from_path(path, by_measure=True, verbose=False)

In [4]:
# Encode them (as entire piece)
me = MidiEncoder()
mdl_enc = {}
for piece in mdl:
    mdl_enc[piece] = me.Encode(mdl[piece].flat, 'pitch_position_duration_strings')

In [5]:
# Encode them (by measure)
me = MidiEncoder()
mdm_enc = {}
for piece in mdm:
    mdm_enc[piece] = {}
    for i, m in enumerate(mdm[piece]):
        mdm_enc[piece][i] = me.Encode(m, 'pitch_position_duration_strings')

# Pre-processing

In [6]:
data = TrainingSet(mdm_enc, by_measure=True, num_notes=1, build_type='next_note')

# Modeling

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
# Sample network, linear model
class Net_1(nn.Module):

    def __init__(self, seq_len, vocab_size):
        super(Net_1, self).__init__()
        self.seq_len = seq_len

        # The linear layer that maps from hidden state space to tag space
        self.out = nn.Linear(self.seq_len, 1)

    def forward(self, note_sequence, state=None):
        x = self.out(note_sequence)
        return x

In [9]:
model = Net_1(1, data.get_vocab_size())
list(model.parameters())

[Parameter containing:
 tensor([[-0.1811]], requires_grad=True),
 Parameter containing:
 tensor([0.7101], requires_grad=True)]

In [10]:
# Params and setup
sequence_length = 10

data = TrainingSet(mdm_enc, by_measure=True, num_notes=sequence_length, build_type='next_note')
model = Net_1(sequence_length, data.get_vocab_size())
loss_function = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [11]:
# Training Loop
for epoch in range(200):
    running_loss = 0
    
    for batch in np.array_split([x for x in range(0, len(data.Xnp))], 20):
        
        # Reset
        model.zero_grad()

        # Data
        inputs = torch.tensor(data.Xnp[batch.tolist()], dtype=torch.float)
        target = torch.tensor([np.array(data.ynp)[batch.tolist()]], dtype=torch.float)

        # Forward
        score = model(inputs)

        # Backward
        loss = loss_function(score, target)
        running_loss += loss
        print(f"Running Loss: {running_loss}                      ", end='\r', flush=True)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch+1} | Running Loss: {running_loss}        ")

Epoch: 1 | Running Loss: 724216.0625                  
Epoch: 2 | Running Loss: 724078.5                     
Epoch: 3 | Running Loss: 723911.6875                 
Epoch: 4 | Running Loss: 724488.1875                 
Epoch: 5 | Running Loss: 724097.0                    
Epoch: 6 | Running Loss: 724488.1875                 
Epoch: 7 | Running Loss: 724097.0                    
Epoch: 8 | Running Loss: 724488.1875                 
Epoch: 9 | Running Loss: 724097.0                    


C:\Users\atawz\miniconda3\envs\pyotr_env_v1\lib\site-packages\torch\nn\modules\loss.py:93: UserWarning: Using a target size (torch.Size([1, 122])) that is different to the input size (torch.Size([122, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
C:\Users\atawz\miniconda3\envs\pyotr_env_v1\lib\site-packages\torch\nn\modules\loss.py:93: UserWarning: Using a target size (torch.Size([1, 121])) that is different to the input size (torch.Size([121, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch: 10 | Running Loss: 724488.1875            
Epoch: 11 | Running Loss: 724097.0                   
Epoch: 12 | Running Loss: 724488.1875                
Epoch: 13 | Running Loss: 724097.0                   
Epoch: 14 | Running Loss: 724488.1875                
Epoch: 15 | Running Loss: 724097.0                   
Epoch: 16 | Running Loss: 724488.1875                
Epoch: 17 | Running Loss: 724097.0                   
Epoch: 18 | Running Loss: 724488.1875                
Epoch: 19 | Running Loss: 724097.0                   
Epoch: 20 | Running Loss: 724488.1875                
Epoch: 21 | Running Loss: 724097.0                   
Epoch: 22 | Running Loss: 724488.1875                
Epoch: 23 | Running Loss: 724097.0                   
Epoch: 24 | Running Loss: 724488.1875                
Epoch: 25 | Running Loss: 724097.0                   
Epoch: 26 | Running Loss: 724488.1875                
Epoch: 27 | Running Loss: 724097.0                   
Epoch: 28 | Running Loss: 724488

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch: 196 | Running Loss: 724488.1875         
Epoch: 197 | Running Loss: 724097.0                  
Epoch: 198 | Running Loss: 724488.1875               
Epoch: 199 | Running Loss: 724097.0                  
Epoch: 200 | Running Loss: 724488.1875               


In [12]:
def scale_to_vocab(output):
    
    min_num = min(output)
    max_num = max(output)
    r = max_num - min_num
    
    new_out = [math.ceil((data.get_vocab_size()-1)*((x-min_num)/r)) for x in output]
    return [data.idx_token_map[x] for x in new_out]

In [21]:
# Predictions
start = data.Xnp[0]
num_preds = 30
pred_input = torch.tensor(start, dtype=torch.float)
out_notes = []

for i in range(0, num_preds):
    next_note = model(pred_input)
    out_notes.append(next_note.item())
    pred_input = torch.cat((pred_input[1:], next_note/out_notes[i-1]))
    
adjusted_output = scale_to_vocab(out_notes)

In [22]:
adjusted_output

['G2:0.0:1.5',
 'B5:0.75:0.25',
 'F3:0.0:1.0',
 'D5:3.25:0.25',
 'B2:2.5:0.25',
 'G#3:1.5:0.25',
 'D3:1.75:0.25',
 'C4:1.0:0.25',
 'C3:0.25:1.75',
 'D3:0.0:2.0',
 'E4:2.25:1.75',
 'C4:0.0:2.0',
 'G4:1.25:0.25',
 'E4:2.25:1.75',
 'E4:2.25:1.75',
 'C5:3.5:0.25',
 'G4:3.25:0.25',
 'E5:3.75:0.25',
 'F5:1.0:0.25',
 'F5:1.75:0.25',
 'D5:2.75:0.25',
 'D5:3.5:0.25',
 'B3:0.0:2.0',
 'D5:3.5:0.25',
 'D5:3.5:0.25',
 'D5:3.5:0.25',
 'A4:3.25:0.25',
 'A4:3.25:0.25',
 'A4:3.25:0.25',
 'A4:3.25:0.25']

In [24]:
md = MidiWriter()
md.Write(adjusted_output, 'pitch_position_duration_strings', 'test3.mid')

<music21.stream.Stream 0x24965c08f08>